<a href="https://colab.research.google.com/github/zi2p/M-machine-learning/blob/main/M33001_%D0%9F%D0%B8%D1%81%D0%B0%D1%80%D0%B5%D0%B2%D0%B0_%22%D0%9A%D0%BE%D0%BD%D1%82%D1%80%D0%BE%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_2_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рубежная контрольная работа

Эту работу выполнять не обязательно. Баллы набранные за работу просуммируются с баллами, которые были набраны за контрольную работу 1 и выставлены как оценка за промежуточную аттестацию "автоматом". 

Сделайте копию ноутбука

Присвойте переменной `v` значение свой табельный номер ИСУ, это нужно для определения вашего варианта. Запустите код в следующей ячейке.

In [ ]:
from datetime import datetime
current_datetime = datetime.now()
print(current_datetime)
v = 311525 # по умолчанию вписан мой номер

2022-12-12 09:45:29.450287


# Общее задание

Нужно решить задачу бинарной классификации, предварительно построив признаковое описание объектов на основе нескольких таблиц. 

Целевая переменная - пол клиента.

В качестве модели нужно использовать нейронную сеть, которую нужно строить с помощью `keras` или `torch` на выбор студента.

# Данные

Для всех последующих заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **transactions, tr_mcc_codes и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк.

Эти данные уже использовались в работе 2. Описание данных тут: https://drive.google.com/file/d/13sn8orjkoGfO1SVZYByXNmZQYFRfmOdp/view?usp=sharing

Скачать данные отсюда: https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L?usp=sharing

## Задание 1

В задании требуется на основе нескольких таблиц с данными сделать признаковое описание объектов. 

Объектами являются клиенты. Клиенты идентифицируются с помощью `customer_id`, которые есть в таблицах **transactions** и **gender_train**. В качестве призаков нужно использовать даннее по категориям транзаций: ```mcc_code``` — mcc-код транзакции есть в таблицах **transactions** и **tr_mcc_codes**


Для выбора варианта, запустите код в следующей ячейке



In [ ]:
print ('Список категорий')
feature_list = ['Одежда (также нужно искать "одежды")', 'Ценные бумаги', 'Магазины косметики и Салоны красоты', 'Образовательные услуги (а также школы, университеты)',
                'Прокат автомобилей', 'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)', 'Видеоигры (искать надо по "видеоигр")',
                'Отели', 'Программирование', 'Ремонт', 'Предсказатели будущего', 'Ювелирные изделия']
if (int(str(v)[4]) % 2) == 0:
  print (feature_list[::2])
else:
  print (feature_list[::-2])


if (int(str(v)[5]) % 2) == 0:
  print ('В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями')
else: 
  print ('В качестве значений признаков используйте сумму по транзакциям с тратами и сумму по транзакциям с поступлениями')

Список категорий
['Одежда (также нужно искать "одежды")', 'Магазины косметики и Салоны красоты', 'Прокат автомобилей', 'Видеоигры (искать надо по "видеоигр")', 'Программирование', 'Предсказатели будущего']
В качестве значений признаков используйте сумму по транзакциям с тратами и сумму по транзакциям с поступлениями


Каждая категория может быть связана с одним или несколькими mcc-кодами. Например, вот несколько mcc-кодов по категории "Одежда": 
* 5621; Готовая женская одежда
* 5641; Детская одежда
* 5651; Одежда для всей семьи

Таким образом получится 12 признаков, то есть шесть категорий, каждая из которых даст два значения: траты и поступления. Например, категория "Одежда" даёт признаки "Одежда траты" и "Одежда поступления".

Сделайте нормализацию значений признаков так, чтобы все означения менялись от 0 до 1. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
tr_mcc_codes = pd.read_csv(filepath_or_buffer = "/content/drive/MyDrive/Data/tr_mcc_codes.csv", sep = ';')
transactions = pd.read_csv(filepath_or_buffer = "/content/drive/MyDrive/Data/transactions.csv", sep = ',', nrows = 1000000)
gender_train = pd.read_csv(filepath_or_buffer = "/content/drive/MyDrive/Data/gender_train.csv", sep = ',')

In [ ]:
def mcc_code_filter(word):
  return tr_mcc_codes[tr_mcc_codes["mcc_description"].str.contains(word, case=False)]

def filtering():
  odejd = mcc_code_filter("одежд")
  cosmetic = mcc_code_filter("космети")
  car = mcc_code_filter("прокат автомобилей")
  game = mcc_code_filter("видеоигр")
  prigramminr = mcc_code_filter("программирование")
  predscaz = mcc_code_filter("предсказатели будущего")
  return [odejd, cosmetic, car, game, prigramminr, predscaz]

def all():
  return mcc_code_filter("(одежд|космети|прокат автомобилей|видеоигр|программирование|предсказатели будущего)")

In [ ]:
_all = all()
transactions = transactions[transactions.mcc_code.isin(_all.mcc_code.to_list())]
type_number = 0
for item in filtering():
  type_number += 1
  transactions.loc[transactions.mcc_code.isin(item["mcc_code"].to_list()), "category"] = type_number
transactions["расходы"] = transactions[transactions["amount"] < 0]["amount"]
transactions["прибыль"] = transactions[transactions["amount"] >= 0]["amount"]
transactions.head()

<ipython-input-11-6e8c6b8edcb3>:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return tr_mcc_codes[tr_mcc_codes["mcc_description"].str.contains(word, case=False)]


,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,category,расходы,прибыль
828,39026145,372 08:45:27,5621,1010,-31442.82,097179,1.0,-31442.82,NaN
829,39026145,372 08:48:29,5621,1010,-11229.58,097179,1.0,-11229.58,NaN
1109,52220754,38 16:48:36,5691,1010,-68433.05,NaN,1.0,-68433.05,NaN
1444,52220754,178 19:11:10,5691,1010,-17742.73,NaN,1.0,-17742.73,NaN
1540,52220754,240 10:44:52,5691,1010,-49163.10,579031,1.0,-49163.10,NaN


In [ ]:
data = transactions.merge(gender_train, on="customer_id").groupby(["customer_id", "category"]).agg({"расходы": "sum", "прибыль": "sum"})
dic = dict()
for row in data.iterrows():
  value = row[1]
  customer_id = row[0][0]
  categoty = row[0][1]
  pos = value['прибыль'] 
  neg = value['расходы']
  if customer_id not in dic:
    dic[customer_id] = {
        **{f'прибыль{int(i)}': 0. for i in range(1, 7, 1)}, 
        **{f'расходы{int(i)}': 0. for i in range(1, 7, 1)}
    }
  dic[customer_id][f'прибыль{categoty}'] = pos
  dic[customer_id][f'расходы{categoty}'] = neg

In [ ]:
df = pd.DataFrame.from_dict(dic, orient='index')
df.index = df.index.rename('customer_id')
all = df.merge(gender_train, on='customer_id')

In [ ]:
all

,customer_id,прибыль1,прибыль2,прибыль3,прибыль4,прибыль5,прибыль6,расходы1,расходы2,расходы3,...,расходы2.0,прибыль5.0,расходы5.0,прибыль6.0,расходы6.0,прибыль4.0,расходы4.0,прибыль3.0,расходы3.0,gender
0,115617,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-473648.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,117556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,189257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-101930.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,293048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-68440.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,361458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,99302313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
752,99412328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-16406.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
753,99683064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-14315.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
754,99717689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
features = ['прибыль1', 'прибыль2', 'прибыль3', 'прибыль4', 'прибыль5', 'прибыль6',
       'расходы1', 'расходы2', 'расходы3', 'расходы4', 'расходы5', 'расходы6']
scaler = StandardScaler()

In [ ]:
scaler.fit(all[features])

StandardScaler()

## Вопросы к заданию 1 (6 баллов)

1. Для каждой категории выведите соответствующие ей mcc-коды с описаниями.
2. Существуют ли пользователи, у которых значения всех признаков не определены?
3. Выведите id пользователя, у которого самое большое количество непустых признаков.

## Ответы:

Для каждой категории выведите соответствующие ей mcc-коды с описаниями.

In [ ]:
result = transactions.merge(tr_mcc_codes, on="mcc_code")[["category", "mcc_description"]]
result.sample(10)

,category,mcc_description
11106,1.0,Различные магазины одежды и аксессуаров
17840,1.0,"Мужская одежда и аксессуары, включая одежду дл..."
14768,2.0,Магазины косметики
1063,1.0,Готовая женская одежда
10556,1.0,Одежда для всей семьи
16133,2.0,Магазины косметики
8064,1.0,"Детская одежда, включая одежду для самых мален..."
12202,1.0,Различные магазины одежды и аксессуаров
18309,1.0,"Спортивная одежда, одежда для верховой езды и ..."
10536,1.0,Одежда для всей семьи


Существуют ли пользователи, у которых значения всех признаков не определены?

In [ ]:
len(all[(all['прибыль1'] == 0) & (all['прибыль2'] == 0) \
& (all['прибыль3'] == 0) & (all['прибыль4'] == 0)  \
& (all['прибыль5'] == 0) & (all['прибыль6'] == 0)  \
& (all['расходы1'] == 0) & (all['расходы2'] == 0) \
& (all['расходы3'] == 0) & (all['расходы4'] == 0) \
& (all['расходы5'] == 0) & (all['расходы6'] == 0)])

756

Выведите id пользователя, у которого самое большое количество непустых признаков.

In [ ]:
frame = all.copy()

for i in range(1, 7):
  frame[f'прибыль{i}'] = frame[f'прибыль{i}'].apply(lambda x: 0 if x == 0 else 1)
  frame[f'расходы{i}'] = frame[f'расходы{i}'].apply(lambda x: 0 if x == 0 else 1)

frame['определенные признаки'] = 0

for i in range(1, 7):
  frame['определенные признаки'] += frame[f'прибыль{i}']
  frame['определенные признаки'] += frame[f'расходы{i}']


frame[['customer_id', 'определенные признаки']].sort_values(by=['определенные признаки'], ascending=False).head(1)

,customer_id,определенные признаки
0,115617,0


# Задание 2 (3 балла)

Модель - многослойная нейронная сеть минимум с двумя скрытыми слоями и с dropout. Количество нейронов в каждом слое выбираете самостоятельно.
Функции активации, метод оптимизации, скорость обучения, вероятность dropout - выбираете самостоятельно

Модель нужно строить с помощью keras или torch

Настроить параметры модели.

## Вопросы к заданию 2

1. Перечислите все гиперпараметры и их значения вот в таком формате (значения приведены примерные, у вас могут быть другие):

* количество эпох: 5
* скорость обучения: 0.0001
* функция активация на скрытых слоях: ReLU
* и т.д.

2. Сколько у вашей модели параметров?

## Ответы:

# Задание 3 (2 балла)


Проверить качество модели не менее, чем на трёх разных метриках. 

## Вопросы к заданию 3

1. Выведите значения метрик.
2. Как улучшить модель?
3. Для чего может понадобиться предсказание пола клиента по его транзакциям?

## Ответы:

После завершения контрольной работы, дайте ссылку на colab в отдельном followup в своей теме на Piazza. Дайте разрешение на его просмотр.

Не забудьте запустить код в последней ячейке для контроля времени выполнения.

In [ ]:
current_datetime = datetime.now()
print(current_datetime)

2022-12-12 10:12:14.667996
